<a href="https://colab.research.google.com/github/MorganButerbaugh/Modeling-the-Optimization-for-Formula-1-Racing/blob/main/F1RaceData%26K_MeansModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding Pre-Race Data in Bahrain (2023)

In [ ]:
!pip install fastf1
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import fastf1 as f1
import fastf1.plotting
import scipy.stats as stats

# Loading Data

In [ ]:
fp1 = f1.get_session(2023, 'Bahrain', 'FP1')
fp1.load()
fp1_laps = fp1.laps

fp2 = f1.get_session(2023, 'Bahrain', 'FP2')
fp2.load()
fp2_laps = fp2.laps

fp3 = f1.get_session(2023, 'Bahrain', 'FP3')
fp3.load()
fp3_laps = fp3.laps

quali = f1.get_session(2023, 'Bahrain', 'Q')
quali.load()
quali_laps = quali.laps

race = f1.get_session(2023, 'Bahrain', 'R')
race.load()
race_laps = race.laps

**Using all 3 Free Practice Sessions and Quailification Round**

# Building the DataFrame

In [ ]:
df = pd.concat([fp1.laps, fp2.laps, fp3.laps, quali.laps])
df.LapTime = df.LapTime.dt.total_seconds()
df = df.reset_index()
df = df[df.IsAccurate == True]
df = df[df.TrackStatus == '1']
df = df[df['Compound'] != 'UNKNOWN']
df = df[df['Compound'] != 'TEST_UNKNOWN']
df = df[df['Compound'] != 'nan']

In [ ]:
colors = {'SOFT': 'red', 'MEDIUM': 'gold', 'HARD': 'grey'}
plt.figure(figsize = (10,4))
sns.scatterplot(data = df, x = 'TyreLife', y = 'LapTime', hue = 'Compound', palette = colors)

In [ ]:
plt.figure(figsize = (8,12))
plt.title('Laptime Distribution per Tyre Compound \n Bahrain (2023)')
sns.boxplot(data = df, x = 'Compound', y = 'LapTime', palette = colors)

# Filtering Data

In [ ]:
### SOFT

sq1 = df[df.Compound == 'SOFT'].LapTime.quantile(0.25)
sq3 = df[df.Compound == 'SOFT'].LapTime.quantile(0.75)
sIQR = sq3-sq1
s_ub = sq3 + 1.5*sIQR
s_lb = sq1 - 1.5*sIQR

### MEDIUM

mq1 = df[df.Compound == 'MEDIUM'].LapTime.quantile(0.25)
mq3 = df[df.Compound == 'MEDIUM'].LapTime.quantile(0.75)
mIQR = mq3-mq1
m_ub = mq3 + 1.5*mIQR
m_lb = mq1 - 1.5*mIQR

### HARD

hq1 = df[df.Compound == 'HARD'].LapTime.quantile(0.25)
hq3 = df[df.Compound == 'HARD'].LapTime.quantile(0.75)
hIQR = hq3-hq1
h_ub = hq3 + 1.5*hIQR
h_lb = hq1 - 1.5*hIQR

In [ ]:
lb_cat = {'SOFT': s_lb, 'MEDIUM': m_lb, 'HARD': h_lb}
ub_cat = {'SOFT': s_ub, 'MEDIUM': m_ub, 'HARD': h_ub}

def filter(row):
  return (row['LapTime'] >= lb_cat[row['Compound']]) and (row['LapTime'] <= ub_cat[row['Compound']])

df = df[df.apply(filter, axis = 1)]

df = df[df.LapTime < 110]
df = df[df.LapTime > 92]

In [ ]:
colors = {'SOFT': 'red', 'MEDIUM': 'gold', 'HARD': 'grey'}
plt.figure(figsize = (10,4))
sns.scatterplot(data = df, x = 'TyreLife', y = 'LapTime', hue = 'Compound', palette = colors)

# Mean Laptime

In [ ]:
df.LapTime.mean()

# Linear Regression

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## Division of Data Based on Tyre Compound

In [ ]:
df_s = df[df.Compound == 'SOFT'][['TyreLife', 'LapTime']]
df_m = df[df.Compound =='MEDIUM'][['TyreLife', 'LapTime']]
df_h = df[df.Compound == 'HARD'][['TyreLife', 'LapTime']]

In [ ]:
xs_train, xs_test, ys_train, ys_test = train_test_split(df_s.TyreLife, df_s.LapTime, test_size = 0.2)
xm_train, xm_test, ym_train, ym_test = train_test_split(df_m.TyreLife, df_m.LapTime, test_size = 0.2)
xh_train, xh_test, yh_train, yh_test = train_test_split(df_h.TyreLife, df_h.LapTime, test_size = 0.2)

xs_train = xs_train.values.reshape(-1, 1)
xs_test = xs_test.values.reshape(-1, 1)
xm_train = xm_train.values.reshape(-1, 1)
xm_test = xm_test.values.reshape(-1, 1)
xh_train = xh_train.values.reshape(-1, 1)
xh_test = xh_test.values.reshape(-1, 1)

ys_train = ys_train.values.reshape(-1, 1)
ys_test = ys_test.values.reshape(-1, 1)
ym_train = ym_train.values.reshape(-1, 1)
ym_test = ym_test.values.reshape(-1, 1)
yh_train = yh_train.values.reshape(-1, 1)
yh_test = yh_test.values.reshape(-1, 1)

## LR for Soft Tyres

In [ ]:
reg_s = linear_model.LinearRegression()
reg_s.fit(xs_train, ys_train)
pred_s = reg_s.predict(xs_test)

# print("Coefficients: \n", reg_s.coef_)
print("Mean Squared Error: %.2f" % mean_squared_error(ys_test, pred_s))
print("R-Squared: %.2f" % r2_score(ys_test, pred_s))
print(f"Training score: {reg_s.score(xs_train, ys_train)*100}%")
print(f"Testing score: {reg_s.score(ys_test, pred_s)*100}%")

In [ ]:
print("Equation: LapTime = {:.2f} * TyreLife + {:.2f}".format(reg_s.coef_[0][0], reg_s.intercept_[0]))

In [ ]:
r_s, p_s = stats.pearsonr(xs_test.ravel(), ys_test.ravel())
r_sq_s = reg_s.score(xs_test, ys_test)

print("Pearson correlation coefficient (r):", r_s)
print("Pearson correlation coefficient squated (R^2):", r_sq_s)
print("P-value:", p_s)

In [ ]:
plt.figure(figsize = (10,6))
sns.regplot(data = df_s, x = 'TyreLife', y = 'LapTime', scatter_kws={'color':'red'}, line_kws={'color':'blue'})
plt.xlabel('Tyre Life (laps)')
plt.ylabel('Lap Time (seconds)')
plt.title('Lap Time vs Tyre Life for SOFT Tyres \n Bahrain (2023)')

r_s, p_s = stats.pearsonr(xs_test.ravel(), ys_test.ravel())
r_sq_s = reg_s.score(xs_test, ys_test)

plt.text(0.79, 0.2, f'r = {r_s:.3f}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)
plt.text(0.79, 0.15, f'R^2 = {r_sq_s:.3f}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)
plt.text(0.79, 0.1, f'p-value = {p_s: .3e}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)

plt.show()

## LR for Medium Tyres

In [ ]:
reg_m = linear_model.LinearRegression()
reg_m.fit(xm_train, ym_train)
pred_m = reg_m.predict(xm_test)

# print("Coefficients: \n", pred_m.coef_)
print("Mean Squared Error: %.2f" % mean_squared_error(ym_test, pred_m))
print("R-Squared: %.2f" % r2_score(ym_test, pred_m))
print(f"Training score: {reg_m.score(xm_train, ym_train)*100}%")
print(f"Testing score: {reg_m.score(ym_test, pred_m)*100}%")

In [ ]:
print("Equation: LapTime = {:.2f} * TyreLife + {:.2f}".format(reg_m.coef_[0][0], reg_m.intercept_[0]))

In [ ]:
r_m, p_m = stats.pearsonr(xm_test.ravel(), ym_test.ravel())
r_sq_m = reg_m.score(xm_test, ym_test)

print("Pearson correlation coefficient (r):", r_m)
print("Pearson correlation coefficient squated (R^2):", r_sq_m)
print("P-value:", p_m)

In [ ]:
plt.figure(figsize = (10,6))
sns.regplot(data = df_m, x = 'TyreLife', y = 'LapTime', scatter_kws={'color':'gold'}, line_kws={'color':'blue'})
plt.xlabel('Tyre Life (laps)')
plt.ylabel('Lap Time (seconds)')
plt.title('Lap Time vs Tyre Life for MEDIUM Tyres \n Bahrain (2023)')

r_m, p_m = stats.pearsonr(xm_test.ravel(), ym_test.ravel())
r_sq_m = reg_m.score(xm_test, ym_test)

plt.text(0.79, 0.2, f'r = {r_m:.3f}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)
plt.text(0.79, 0.15, f'R^2 = {r_sq_m:.3f}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)
plt.text(0.79, 0.1, f'p-value = {p_m: .3e}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)

plt.show()

## LR for Hard Tyres

In [ ]:
reg_h = linear_model.LinearRegression()
reg_h.fit(xh_train, yh_train)
pred_h = reg_h.predict(xh_test)

# print("Coefficients: \n", reg_h.coef_)
print("Mean Squared Error: %.2f" % mean_squared_error(yh_test, pred_h))
print("R-Squared: %.2f" % r2_score(yh_test, pred_h))
print(f"Training score: {reg_h.score(xh_train, yh_train)*100}%")
print(f"Testing score: {reg_h.score(yh_test, pred_h)*100}%")

In [ ]:
print("Equation: LapTime = {:.2f} * TyreLife + {:.2f}".format(reg_h.coef_[0][0], reg_h.intercept_[0]))

In [ ]:
r_h, p_h = stats.pearsonr(xh_test.ravel(), yh_test.ravel())
r_sq_h = reg_h.score(xh_test, yh_test)

print("Pearson correlation coefficient (r):", r_h)
print("Pearson correlation coefficient squated (R^2):", r_sq_h)
print("P-value:", p_h)

In [ ]:
plt.figure(figsize = (10,6))
sns.regplot(data = df_h, x = 'TyreLife', y = 'LapTime', scatter_kws={'color':'grey'}, line_kws={'color':'blue'})
plt.xlabel('Tyre Life (laps)')
plt.ylabel('Lap Time (seconds)')
plt.title('Lap Time vs Tyre Life for HARD Tyres \n Bahrain (2023)')

r_h, p_h = stats.pearsonr(xh_test.ravel(), yh_test.ravel())
r_sq_h = reg_h.score(xh_test, yh_test)

plt.text(0.79, 0.2, f'r = {r_h:.3f}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)
plt.text(0.79, 0.15, f'R^2 = {r_sq_h:.3f}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)
plt.text(0.79, 0.1, f'p-value = {p_h: .3e}', horizontalalignment='left', verticalalignment='center', transform=plt.gca().transAxes)

plt.show()

-----------

# Mean PitStop Time Estimation

In [ ]:
pits = race.laps[['PitInTime', 'PitOutTime']]
pits = pits.reset_index()
pits = pits.dropna(how = 'all')
pits['PitOutTime'] = pits['PitOutTime'].shift(-1)
pits = pits.dropna()
pits['PitTime'] = (pits.PitOutTime - pits.PitInTime).dt.total_seconds()

q1 = pits.PitTime.quantile(0.25)
q3 = pits.PitTime.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr)
upper_bound = q3 + (1.5 * iqr)

pits = pits[(pits.PitTime >= lower_bound) & (pits.PitTime <= upper_bound)]

In [ ]:
print(pits.PitTime.mean())

--------------

# Top Race Time

In [ ]:
top_df = race.laps[race.laps.Driver == 'VER']

top_df.LapTime = top_df.LapTime.dt.total_seconds()

In [ ]:
top_df.LapTime.sum()

---------------

# Tyre Compound Graphs

In [ ]:
fp1_age = fp1_laps[fp1_laps.Driver.isin(['VER', 'SAI', 'BOT', 'GAS'])].groupby(['Driver', 'Stint', 'Compound']).TyreLife.min().reset_index()
fp2_age = fp2_laps[fp2_laps.Driver.isin(['VER', 'SAI', 'BOT', 'GAS'])].groupby(['Driver', 'Stint', 'Compound']).TyreLife.min().reset_index()
fp3_age = fp3_laps[fp3_laps.Driver.isin(['VER', 'SAI', 'BOT', 'GAS'])].groupby(['Driver', 'Stint', 'Compound']).TyreLife.min().reset_index()
quali_age = quali_laps[quali_laps.Driver.isin(['VER', 'SAI', 'BOT', 'GAS'])].groupby(['Driver', 'Stint', 'Compound']).TyreLife.min().reset_index()

fp1_age = fp1_age[fp1_age.TyreLife == 1.0]
fp1_age['Session'] = 'FP1'

fp2_age = fp2_age[fp2_age.TyreLife == 1.0]
fp2_age['Session'] = 'FP2'

fp3_age = fp3_age[fp3_age.TyreLife == 1.0]
fp3_age['Session'] = 'FP3'

quali_age = quali_age[quali_age.TyreLife == 1.0]
quali_age['Session'] = 'Q'

tot_tyres = pd.concat([fp1_age, fp2_age, fp3_age, quali_age]).sort_values(by = ['Driver', 'Session']).reset_index()

In [ ]:
tot_tyres

In [ ]:
fp1_drivers = fp1.drivers
fp2_drivers = fp2.drivers
fp3_drivers = fp3.drivers
quali_drivers = quali.drivers

In [ ]:
fp1_drivers = [fp1.get_driver(driver)["Abbreviation"] for driver in fp1_drivers]

fp2_drivers = [fp2.get_driver(driver)["Abbreviation"] for driver in fp2_drivers]

fp3_drivers = [fp3.get_driver(driver)["Abbreviation"] for driver in fp3_drivers]

quali_drivers = [quali.get_driver(driver)["Abbreviation"] for driver in quali_drivers]

In [ ]:
fp1_stints = fp1_laps[["Driver", "Stint", "Compound", "LapNumber"]]
fp1_stints = fp1_stints.groupby(["Driver", "Stint", "Compound"])
fp1_stints = fp1_stints.count().reset_index()

fp2_stints = fp2_laps[["Driver", "Stint", "Compound", "LapNumber"]]
fp2_stints = fp2_stints.groupby(["Driver", "Stint", "Compound"])
fp2_stints = fp2_stints.count().reset_index()

fp3_stints = fp3_laps[["Driver", "Stint", "Compound", "LapNumber"]]
fp3_stints = fp3_stints.groupby(["Driver", "Stint", "Compound"])
fp3_stints = fp3_stints.count().reset_index()

quali_stints = quali_laps[["Driver", "Stint", "Compound", "LapNumber"]]
quali_stints = quali_stints.groupby(["Driver", "Stint", "Compound"])
quali_stints = quali_stints.count().reset_index()

In [ ]:
fp1_stints = fp1_stints.rename(columns={"LapNumber": "StintLength"})
fp2_stints = fp2_stints.rename(columns={"LapNumber": "StintLength"})
fp3_stints = fp3_stints.rename(columns={"LapNumber": "StintLength"})
quali_stints = quali_stints.rename(columns={"LapNumber": "StintLength"})

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5))

for driver in fp1_drivers:
    driver_stints = fp1_stints.loc[fp1_stints["Driver"] == driver]

    previous_stint_end = 0
    for idx, row in driver_stints.iterrows():
        # each row contains the compound name and stint length
        # we can use these information to draw horizontal bars
        plt.barh(
            y=driver,
            width=row["StintLength"],
            left=previous_stint_end,
            color=fastf1.plotting.COMPOUND_COLORS[row["Compound"]],
            edgecolor="black",
            fill=True
        )

        previous_stint_end += row["StintLength"]

plt.title("2023 FP1 Bahrain Strategies")
plt.xlabel("Lap Number")
plt.grid(False)
# invert the y-axis so drivers that finish higher are closer to the top
ax.invert_yaxis()

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5))

for driver in fp2_drivers:
    driver_stints = fp2_stints.loc[fp2_stints["Driver"] == driver]

    previous_stint_end = 0
    for idx, row in driver_stints.iterrows():
        # each row contains the compound name and stint length
        # we can use these information to draw horizontal bars
        plt.barh(
            y=driver,
            width=row["StintLength"],
            left=previous_stint_end,
            color=fastf1.plotting.COMPOUND_COLORS[row["Compound"]],
            edgecolor="black",
            fill=True
        )

        previous_stint_end += row["StintLength"]

plt.title("2023 FP2 Bahrain Strategies")
plt.xlabel("Lap Number")
plt.grid(False)
# invert the y-axis so drivers that finish higher are closer to the top
ax.invert_yaxis()

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5))

for driver in fp3_drivers:
    driver_stints = fp3_stints.loc[fp3_stints["Driver"] == driver]

    previous_stint_end = 0
    for idx, row in driver_stints.iterrows():
        # each row contains the compound name and stint length
        # we can use these information to draw horizontal bars
        plt.barh(
            y=driver,
            width=row["StintLength"],
            left=previous_stint_end,
            color=fastf1.plotting.COMPOUND_COLORS[row["Compound"]],
            edgecolor="black",
            fill=True
        )

        previous_stint_end += row["StintLength"]

plt.title("2023 FP3 Bahrain Strategies")
plt.xlabel("Lap Number")
plt.grid(False)
# invert the y-axis so drivers that finish higher are closer to the top
ax.invert_yaxis()

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5))

for driver in quali_drivers:
    driver_stints = quali_stints.loc[quali_stints["Driver"] == driver]

    previous_stint_end = 0
    for idx, row in driver_stints.iterrows():
        # each row contains the compound name and stint length
        # we can use these information to draw horizontal bars
        plt.barh(
            y=driver,
            width=row["StintLength"],
            left=previous_stint_end,
            color=fastf1.plotting.COMPOUND_COLORS[row["Compound"]],
            edgecolor="black",
            fill=True
        )

        previous_stint_end += row["StintLength"]

plt.title("2023 Quali Bahrain Strategies")
plt.xlabel("Lap Number")
plt.grid(False)
# invert the y-axis so drivers that finish higher are closer to the top
ax.invert_yaxis()

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.tight_layout()
plt.show()

------------

# K-Means Model On All F1 Tracks for 2022 Qualifications

In [ ]:
!pip install fastf1
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import fastf1 as f1
import scipy.stats as stats

In [ ]:
q_bar = f1.get_session(2022, 'Bahrain', 'Q')
q_bar.load()

q_jed = f1.get_session(2022, 'Saudi', 'Q')
q_jed.load()

q_aust = f1.get_session(2022, 'Australia', 'Q')
q_aust.load()

q_imo = f1.get_session(2022, 'Imola', 'Q')
q_imo.load()

q_mia = f1.get_session(2022, 'Miami', 'Q')
q_mia.load()

q_spa = f1.get_session(2022, 'Spain', 'Q')
q_spa.load()

q_mon = f1.get_session(2022, 'Monaco', 'Q')
q_mon.load()

q_bak = f1.get_session(2022, 'Baku', 'Q')
q_bak.load()

q_can = f1.get_session(2022, 'Canada', 'Q')
q_can.load()

q_uk = f1.get_session(2022, 'Silverstone', 'Q')
q_uk.load()

q_aus = f1.get_session(2022, 'Austria', 'Q')
q_aus.load()

q_fra = f1.get_session(2022, 'France', 'Q')
q_fra.load()

q_hun = f1.get_session(2022, 'Hungary', 'Q')
q_hun.load()

q_bel = f1.get_session(2022, 'Belgium', 'Q')
q_bel.load()

q_ned = f1.get_session(2022, 'Netherlands', 'Q')
q_ned.load()

q_ita = f1.get_session(2022, 'Monza', 'Q')
q_ita.load()

q_sin = f1.get_session(2022, 'Singapore', 'Q')
q_sin.load()

q_jap = f1.get_session(2022, 'Japan', 'Q')
q_jap.load()

q_usa = f1.get_session(2022, 'Austin', 'Q')
q_usa.load()

q_mex = f1.get_session(2022, 'Mexico', 'Q')
q_mex.load()

q_bra = f1.get_session(2022, 'Brazil', 'Q')
q_bra.load()

q_abu = f1.get_session(2022, 'Abu Dhabi', 'Q')
q_abu.load()

In [ ]:
quali = [q_bar, q_jed, q_aust, q_imo, q_mia, q_spa, q_mon, q_bak, q_can, q_uk, q_aus,
         q_fra, q_hun, q_bel, q_ned, q_ita, q_sin, q_jap, q_usa, q_mex, q_bra, q_abu]

In [ ]:
avg_speed = []
top_speed = []
throttle = []
wcss = []

In [ ]:
for i in range(len(quali)):

    q = quali[i].laps.pick_fastest().get_telemetry()

    throttle.append((q[q.Throttle >= 95.0].Throttle.count()/q.Throttle.count()).round(2))
    avg_speed.append(q.Speed.mean().round(2))
    top_speed.append(q.Speed.max())

In [ ]:
event = ['Bahrain', 'Saudi Arabia', 'Australia', 'Emilia Romagna', 'Miami', 'Spain', 'Monaco', 'Azerbaijan', 'Canada', 'Britain',
         'Austria', 'France', 'Hungary', 'Belgium', 'Netherlands', 'Monza', 'Singapore', 'Japan', 'Texas', 'Mexico',
         'Brazil', 'UAE']

In [ ]:
nary = {'Event': event, 'Top Speed': top_speed, 'AVG Speed': avg_speed, 'Full Throttle %' : throttle}

In [ ]:
df = pd.DataFrame(nary)
df = df[df.Event != 'Britain']
df

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(df.iloc[:, 1:])
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=0)
y_pred = kmeans.fit_predict(df.iloc[:, 1:])

df['Cluster'] = kmeans.labels_


group1 = df[df.Cluster == 0]
group2 = df[df.Cluster == 1]
group3 = df[df.Cluster == 2]
group4 = df[df.Cluster == 3]

group1.mean(numeric_only=True).round(2)
group2.mean(numeric_only=True).round(2)
group3.mean(numeric_only=True).round(2)
group4.mean(numeric_only=True).round(2)
group2